In [ ]:
import pandas as pd
import sklearn.ensemble
import seaborn as sns
import numpy as np

In [ ]:
# Get data
import os
import urllib.request

if not os.path.exists('./data/raw/smoking_input.csv') or not os.path.exists('./data/raw/smoking_targets.csv'):
    os.system('mkdir -p data/raw')
    urllib.request.urlretrieve("http://ifestos.cse.sc.edu/datasets/smoking_data.tar.gz", "data/smoking_data.tar.gz")

    os.system('tar -xzvf data/smoking_data.tar.gz -C data/raw/ --strip-components=1')

In [ ]:
# Load data

raw_input = pd.read_csv('data/raw/smoking_input.csv', header=None)
targets = pd.read_csv('data/raw/smoking_targets.csv', header=None)

In [ ]:
# Visualize data

t = targets.copy(deep=True)
t[0] = t[0].apply(lambda x: x+10)

targets_to_plot = t[0].to_numpy()
targets_to_plot = np.reshape(targets_to_plot, [len(targets_to_plot),1])
# targets_to_plot = np.pad(targets_to_plot,((0,0),(0,199)), 'mean').flatten()
targets_to_plot = np.pad(targets_to_plot,((0,0),(0,99)), 'mean').flatten()


to_plot = pd.DataFrame()
to_plot['x'] = raw_input.loc[:, 0:99].to_numpy().flatten()
to_plot['y'] = raw_input.loc[:, 100:199].to_numpy().flatten()
to_plot['z'] = raw_input.loc[:, 200:299].to_numpy().flatten()
# to_plot['x'] = rows = np.pad(raw_input.loc[:, 0:99].to_numpy(), ((0,0),(50,50)), 'constant', constant_values=(0)).flatten()
# to_plot['y'] = rows = np.pad(raw_input.loc[:, 100:199].to_numpy(), ((0,0),(50,50)), 'constant', constant_values=(0)).flatten()
# to_plot['z'] = rows = np.pad(raw_input.loc[:, 200:299].to_numpy(), ((0,0),(50,50)), 'constant', constant_values=(0)).flatten()

to_plot['label'] = targets_to_plot

In [ ]:
sns.lineplot(to_plot[:2000], dashes=None)

In [ ]:
# Clean Data
targets.columns = ["labels"]
df = pd.concat([raw_input, targets], axis=1)
df = df.fillna(method='bfill')

In [ ]:
# Split data into train and test
from sklearn.model_selection import train_test_split

(X_train, X_test, y_train, y_test) = train_test_split(df.drop(['labels'], axis=1), df['labels'], test_size=0.50, stratify=df['labels'])

In [ ]:
# Train Random Forest Classifier

rf = sklearn.ensemble.RandomForestClassifier(n_estimators=100, max_depth=3)

rf.fit(X_train, y_train)

In [ ]:

sns.lineplot(rf.feature_importances_)

In [ ]:
# Test Classifier

acc = rf.score(X_test, y_test)

In [ ]:
# Heat Map
pred = rf.predict(X_test)

fp = tp = fn = tn = 0

for i,p in enumerate(pred):
    if p == 1:
        if p == y_test.iloc[i]:
            tp += 1
        else:
            fp += 1
    else:   
        if p == y_test.iloc[i]:
            tn += 1
        else:
            fn += 1

sns.heatmap([[tn, fp],[fn, tp]], annot=True)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred, normalize="true")
sns.heatmap(cm, annot=True)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import recall_score


# balanced_accuracy_score(y_test, y_pred=pred)
recall_score(y_test, y_pred=pred, average='macro')
